# Model Training
Train XGBoost model for NIFTY prediction

In [ ]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../backend')

from app.ml.features import FeatureEngine
from app.ml.train import ModelTrainer
from app.ml.backtest import Backtester

In [ ]:
# Load and prepare data
dates = pd.date_range('2023-01-01', '2023-12-31', freq='1min')
df = pd.DataFrame({
    'open': 19500 + np.random.randn(len(dates)).cumsum() * 10,
    'high': 19520 + np.random.randn(len(dates)).cumsum() * 10,
    'low': 19480 + np.random.randn(len(dates)).cumsum() * 10,
    'close': 19500 + np.random.randn(len(dates)).cumsum() * 10,
    'volume': np.random.randint(1000, 10000, len(dates))
}, index=dates)

print(f"Data shape: {df.shape}")

In [ ]:
# Train model
trainer = ModelTrainer()
model, auc = trainer.train_xgboost(df)
print(f"Model trained with AUC: {auc:.4f}")

In [ ]:
# Backtest
df_features = FeatureEngine.prepare_features(df)
predictions = model.predict(df_features[trainer.feature_cols])
signals = pd.Series(predictions, index=df_features.index)

backtester = Backtester(initial_capital=100000)
results = backtester.run(df_features, signals)

print("\nBacktest Results:")
for key, value in results.items():
    if key != 'trades':
        print(f"{key}: {value}")